In [11]:
"""Cell 1: Setup and Imports"""
import sys
import os
import time
import warnings
import numpy as np
from IPython.display import clear_output

warnings.filterwarnings('ignore')

# Ensure src is in path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Clear all qectostim module cache BEFORE importing (fresh start)
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

# Import testing utilities from qectostim.testing
from qectostim.testing import (
    clear_qectostim_modules,
    test_composite_construction,
    test_code_circuit,
    test_decoder_on_code,
    load_all_decoders,
    STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL,
)

# Import qectostim
from qectostim.codes import discover_all_codes
from qectostim.codes.composite import (
    ConcatenatedTopologicalCSSCode,
    HomologicalProductCode,
    DualCode,
)
from qectostim.codes.abstract_css import CSSCodeWithComplex
from qectostim.decoders import PyMatchingDecoder

# Load all available decoders
decoder_classes = load_all_decoders()

# Try to import ConcatenatedDecoder
try:
    from qectostim.decoders.concatenated_decoder import ConcatenatedDecoder
    HAS_CONCAT_DECODER = True
except ImportError as e:
    HAS_CONCAT_DECODER = False
    print(f"Note: ConcatenatedDecoder not available: {e}")

print("✓ All imports successful")
print(f"  ConcatenatedTopologicalCSSCode: {ConcatenatedTopologicalCSSCode}")
print(f"  HomologicalProductCode: {HomologicalProductCode}")
print(f"  DualCode: {DualCode}")
print(f"  Decoders loaded: {list(decoder_classes.keys())}")
print(f"  ConcatenatedDecoder available: {HAS_CONCAT_DECODER}")

✓ All imports successful
  ConcatenatedTopologicalCSSCode: <class 'qectostim.codes.composite.concatenated.ConcatenatedTopologicalCSSCode'>
  HomologicalProductCode: <class 'qectostim.codes.composite.homological_product.HomologicalProductCode'>
  DualCode: <class 'qectostim.codes.composite.dual.DualCode'>
  Decoders loaded: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius']
  ConcatenatedDecoder available: True


In [12]:
"""Cell 2: Discover Building Block Codes"""
clear_output(wait=True)

print("="*70)
print("DISCOVERING BUILDING BLOCK CODES")
print("="*70)

# Discover codes - use smaller size for composite building
all_codes = discover_all_codes(
    max_qubits=50,  # Smaller for composition
    include_qldpc=True,
    include_subsystem=False,  # Skip subsystem for CSS operations
    include_floquet=False,    # Skip floquet for CSS operations
    timeout_per_code=3.0,
)

# Filter to CSS codes only (required for concatenation and homological product)
css_codes = {}
for name, code in all_codes.items():
    if hasattr(code, 'hx') and hasattr(code, 'hz'):
        # Additional check: must have k > 0
        if code.k > 0:
            css_codes[name] = code

print(f"\nTotal discovered: {len(all_codes)}")
print(f"CSS codes with k>0: {len(css_codes)}")

# Categorize for smart pairing
small_codes = {}   # n <= 10, good for inner codes
medium_codes = {}  # 10 < n <= 30
surface_codes = {} # Surface-like, good for outer codes
other_codes = {}

for name, code in css_codes.items():
    n = code.n
    if 'Surface' in name or 'Toric' in name:
        surface_codes[name] = code
    elif n <= 10:
        small_codes[name] = code
    elif n <= 30:
        medium_codes[name] = code
    else:
        other_codes[name] = code

print(f"\nBy size:")
print(f"  Small (n≤10): {len(small_codes)}")
print(f"  Medium (10<n≤30): {len(medium_codes)}")
print(f"  Surface-like: {len(surface_codes)}")
print(f"  Other: {len(other_codes)}")

# List codes
print("\n" + "-"*70)
print(f"{'Code Name':<40} {'n':>4} {'k':>3} {'d':>3}")
print("-"*70)
for name, code in sorted(css_codes.items(), key=lambda x: x[1].n)[:20]:
    d = code.metadata.get('distance', '?')
    print(f"{name:<40} {code.n:>4} {code.k:>3} {d:>3}")
if len(css_codes) > 20:
    print(f"... and {len(css_codes) - 20} more")

DISCOVERING BUILDING BLOCK CODES

Total discovered: 38
CSS codes with k>0: 33

By size:
  Small (n≤10): 13
  Medium (10<n≤30): 6
  Surface-like: 8
  Other: 6

----------------------------------------------------------------------
Code Name                                   n   k   d
----------------------------------------------------------------------
Repetition_3                                3   1   3
FourQubit422_[[4,2,2]]                      4   2   2
Repetition_5                                5   1   5
C6                                          6   2   2
Steane_713                                  7   1   3
Hamming_CSS_7                               7   1   3
Repetition_7                                7   1   7
TriangularColour_d3                         7   1   3
Code_832                                    8   3   2
HexagonalColour_d2                          8   2   2
Shor_91                                     9   1   3
RotatedSurface_[[9,1,3]]                    9   1  

In [13]:
"""Cell 3: Test Concatenation - All Combinations"""
clear_output(wait=True)

print("="*70)
print("CONCATENATION TESTS")
print("="*70)
print("\nTesting ConcatenatedTopologicalCSSCode(outer, inner)")
print("Expected: n_concat = n_outer × n_inner\n")

concat_results = []

# Generate test pairs: use small codes as inner, surface/medium as outer
# Limit to avoid combinatorial explosion
inner_candidates = list(small_codes.items())[:5]  # Top 5 small codes
outer_candidates = list(surface_codes.items())[:3] + list(medium_codes.items())[:2]  # 5 outer codes

test_pairs = []
for outer_name, outer_code in outer_candidates:
    for inner_name, inner_code in inner_candidates:
        # Skip if would be too large
        expected_n = outer_code.n * inner_code.n
        if expected_n <= 200:  # Keep manageable
            test_pairs.append((outer_name, outer_code, inner_name, inner_code))

print(f"Testing {len(test_pairs)} concatenation pairs...\n")

for i, (outer_name, outer_code, inner_name, inner_code) in enumerate(test_pairs):
    print(f"[{i+1}/{len(test_pairs)}] {outer_name} ∘ {inner_name}...", end=' ', flush=True)
    
    result = {
        'outer': outer_name,
        'inner': inner_name,
        'status': 'UNKNOWN',
        'n': None,
        'k': None,
        'circuit_ok': False,
        'error': None,
    }
    
    try:
        # Test construction
        success, concat, msg = test_composite_construction(
            outer_code, inner_code, ConcatenatedTopologicalCSSCode
        )
        
        if success and concat:
            result['n'] = concat.n
            result['k'] = concat.k
            
            # Verify expected n
            expected_n = outer_code.n * inner_code.n
            if concat.n != expected_n:
                result['status'] = 'WARN'
                result['error'] = f"n={concat.n} != expected {expected_n}"
            else:
                # Test circuit generation
                circ_ok, circuit, circ_msg = test_code_circuit(concat)
                result['circuit_ok'] = circ_ok
                
                if circ_ok:
                    result['status'] = 'OK'
                else:
                    result['status'] = 'WARN'
                    result['error'] = circ_msg[:30]
        else:
            result['status'] = 'FAIL'
            result['error'] = msg[:40]
            
    except Exception as e:
        result['status'] = 'FAIL'
        result['error'] = f"{type(e).__name__}: {str(e)[:30]}"
    
    concat_results.append(result)
    
    # Print status
    if result['status'] == 'OK':
        print(f"{STATUS_OK} n={result['n']}, k={result['k']}")
    elif result['status'] == 'WARN':
        print(f"{STATUS_WARN} {result['error']}")
    else:
        print(f"{STATUS_FAIL} {result['error']}")

# Summary
ok = sum(1 for r in concat_results if r['status'] == 'OK')
warn = sum(1 for r in concat_results if r['status'] == 'WARN')
fail = sum(1 for r in concat_results if r['status'] == 'FAIL')

print("\n" + "-"*70)
print(f"CONCATENATION SUMMARY: {ok} {STATUS_OK} | {warn} {STATUS_WARN} | {fail} {STATUS_FAIL}")

CONCATENATION TESTS

Testing ConcatenatedTopologicalCSSCode(outer, inner)
Expected: n_concat = n_outer × n_inner

Testing 24 concatenation pairs...

[1/24] RotatedSurface_[[9,1,3]] ∘ FourQubit422_[[4,2,2]]... ✓ n=36, k=10
[2/24] RotatedSurface_[[9,1,3]] ∘ C6... ✓ n=36, k=10
[2/24] RotatedSurface_[[9,1,3]] ∘ C6... ✓ n=54, k=10
[3/24] RotatedSurface_[[9,1,3]] ∘ Steane_713... ✓ n=54, k=10
[3/24] RotatedSurface_[[9,1,3]] ∘ Steane_713... ✓ n=63, k=1
[4/24] RotatedSurface_[[9,1,3]] ∘ Shor_91... ✓ n=63, k=1
[4/24] RotatedSurface_[[9,1,3]] ∘ Shor_91... ✓ n=81, k=1
[5/24] RotatedSurface_[[9,1,3]] ∘ Hamming_CSS_7... ✓ n=81, k=1
[5/24] RotatedSurface_[[9,1,3]] ∘ Hamming_CSS_7... ✓ n=63, k=1
[6/24] RotatedSurface_[[25,1,5]] ∘ FourQubit422_[[4,2,2]]... ✓ n=63, k=1
[6/24] RotatedSurface_[[25,1,5]] ∘ FourQubit422_[[4,2,2]]... ✓ n=100, k=26
[7/24] RotatedSurface_[[25,1,5]] ∘ C6... ✓ n=100, k=26
[7/24] RotatedSurface_[[25,1,5]] ∘ C6... ✓ n=150, k=26
[8/24] RotatedSurface_[[25,1,5]] ∘ Steane_713... ✓ n=

In [14]:
"""Cell 4: Test Homological Product"""
clear_output(wait=True)

print("="*70)
print("HOMOLOGICAL PRODUCT TESTS")
print("="*70)
print("\nTesting HomologicalProductCode(code_a, code_b)")
print("Requires codes with chain complexes (topological codes)\n")

# Check which codes have chain complexes - check for attribute directly
codes_with_complex = {}
for name, code in css_codes.items():
    # Check if code has a chain_complex attribute that is not None
    if hasattr(code, 'chain_complex') and code.chain_complex is not None:
        codes_with_complex[name] = code

print(f"CSS codes with chain_complex attribute: {len(codes_with_complex)}")
if codes_with_complex:
    print(f"  Found: {list(codes_with_complex.keys())[:5]}...")

if len(codes_with_complex) < 2:
    # Try importing specific topological codes directly
    print("\nInsufficient codes with chain complexes found in discovery.")
    print("Importing topological codes directly...")
    
    try:
        from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
        from qectostim.codes.surface.toric_code import ToricCode
        
        # Create codes and verify they have chain complexes
        rs3 = RotatedSurfaceCode(distance=3)
        if hasattr(rs3, 'chain_complex') and rs3.chain_complex is not None:
            codes_with_complex['RotatedSurface_d3'] = rs3
            print(f"  ✓ Added RotatedSurface_d3 (n={rs3.n})")
        
        tc3 = ToricCode(Lx=3, Ly=3)
        if hasattr(tc3, 'chain_complex') and tc3.chain_complex is not None:
            codes_with_complex['ToricCode_3x3'] = tc3
            print(f"  ✓ Added ToricCode_3x3 (n={tc3.n})")
            
    except Exception as e:
        print(f"  ✗ Failed: {type(e).__name__}: {e}")
    
    try:
        from qectostim.codes.base.repetition import RepetitionCode
        rep3 = RepetitionCode(N=3)
        if hasattr(rep3, 'chain_complex') and rep3.chain_complex is not None:
            codes_with_complex['Repetition_3'] = rep3
            print(f"  ✓ Added Repetition_3 (n={rep3.n})")
    except Exception as e:
        print(f"  Repetition: {type(e).__name__}: {e}")

hom_results = []

if len(codes_with_complex) >= 1:
    # Generate pairs for homological product - including self-products
    complex_list = list(codes_with_complex.items())[:4]
    
    hom_pairs = []
    for i, (name_a, code_a) in enumerate(complex_list):
        for j, (name_b, code_b) in enumerate(complex_list):
            if j >= i:  # Include self-products (A ⊗ A)
                # Check size constraint
                if code_a.n * code_b.n <= 500:
                    hom_pairs.append((name_a, code_a, name_b, code_b))
    
    print(f"\nTesting {len(hom_pairs)} homological product pairs...\n")
    
    for i, (name_a, code_a, name_b, code_b) in enumerate(hom_pairs):
        print(f"[{i+1}/{len(hom_pairs)}] {name_a} ⊗ {name_b}...", end=' ', flush=True)
        
        result = {
            'code_a': name_a,
            'code_b': name_b,
            'status': 'UNKNOWN',
            'n': None,
            'k': None,
            'error': None,
        }
        
        try:
            # Try direct construction first
            product = HomologicalProductCode(code_a, code_b)
            result['n'] = product.n
            result['k'] = product.k
            
            if product.k > 0:
                result['status'] = 'OK'
            else:
                result['status'] = 'WARN'
                result['error'] = f"k={product.k} (expected k>0)"
                
        except Exception as e:
            result['status'] = 'FAIL'
            result['error'] = f"{type(e).__name__}: {str(e)[:40]}"
        
        hom_results.append(result)
        
        if result['status'] == 'OK':
            print(f"{STATUS_OK} n={result['n']}, k={result['k']}")
        elif result['status'] == 'WARN':
            print(f"{STATUS_WARN} n={result['n']}, {result['error']}")
        else:
            print(f"{STATUS_FAIL} {result['error']}")
else:
    print("\n⚠️ Skipping homological product tests - no codes with chain complexes found.")

# Summary
if hom_results:
    ok = sum(1 for r in hom_results if r['status'] == 'OK')
    warn = sum(1 for r in hom_results if r['status'] == 'WARN')
    fail = sum(1 for r in hom_results if r['status'] == 'FAIL')
    print("\n" + "-"*70)
    print(f"HOMOLOGICAL PRODUCT SUMMARY: {ok} {STATUS_OK} | {warn} {STATUS_WARN} | {fail} {STATUS_FAIL}")

HOMOLOGICAL PRODUCT TESTS

Testing HomologicalProductCode(code_a, code_b)
Requires codes with chain complexes (topological codes)

CSS codes with chain_complex attribute: 17
  Found: ['FourQubit422_[[4,2,2]]', 'Steane_713', 'Shor_91', 'Hamming_CSS_7', 'Repetition_3']...

Testing 10 homological product pairs...

[1/10] FourQubit422_[[4,2,2]] ⊗ FourQubit422_[[4,2,2]]... ✓ n=18, k=4
[2/10] FourQubit422_[[4,2,2]] ⊗ Steane_713... ✓ n=18, k=4
[2/10] FourQubit422_[[4,2,2]] ⊗ Steane_713... ✓ n=34, k=2
[3/10] FourQubit422_[[4,2,2]] ⊗ Shor_91... ✓ n=34, k=2
[3/10] FourQubit422_[[4,2,2]] ⊗ Shor_91... ✓ n=44, k=2
[4/10] FourQubit422_[[4,2,2]] ⊗ Hamming_CSS_7... ✓ n=44, k=2
[4/10] FourQubit422_[[4,2,2]] ⊗ Hamming_CSS_7... ✓ n=34, k=2
[5/10] Steane_713 ⊗ Steane_713... ✓ n=34, k=2
[5/10] Steane_713 ⊗ Steane_713... ✓ n=67, k=1
[6/10] Steane_713 ⊗ Shor_91... ✓ n=67, k=1
[6/10] Steane_713 ⊗ Shor_91... ✓ n=87, k=1
[7/10] Steane_713 ⊗ Hamming_CSS_7... ✓ n=87, k=1
[7/10] Steane_713 ⊗ Hamming_CSS_7... ✓ n=6

In [15]:
"""Cell 5: Multi-Level Concatenation"""
clear_output(wait=True)

print("="*70)
print("MULTI-LEVEL CONCATENATION TESTS")
print("="*70)
print("\nTesting nested concatenation: (outer ∘ middle) ∘ inner")
print("Demonstrates composability of composite operations\n")

multi_results = []

# Select small codes for multi-level (to keep size manageable)
small_list = list(small_codes.items())[:4]

# Test cases for 2-level concatenation
multilevel_cases = []

# Pattern: (A ∘ B) ∘ C
if len(small_list) >= 3:
    for i in range(min(3, len(small_list))):
        outer = small_list[i]
        middle = small_list[(i+1) % len(small_list)]
        inner = small_list[(i+2) % len(small_list)]
        
        # Check size
        expected_n = outer[1].n * middle[1].n * inner[1].n
        if expected_n <= 500:
            multilevel_cases.append((outer, middle, inner))

print(f"Testing {len(multilevel_cases)} multi-level concatenations...\n")

for i, (outer, middle, inner) in enumerate(multilevel_cases):
    outer_name, outer_code = outer
    middle_name, middle_code = middle
    inner_name, inner_code = inner
    
    desc = f"({outer_name} ∘ {middle_name}) ∘ {inner_name}"
    print(f"[{i+1}/{len(multilevel_cases)}] {desc}...", end=' ', flush=True)
    
    result = {
        'desc': desc,
        'status': 'UNKNOWN',
        'n': None,
        'k': None,
        'error': None,
    }
    
    try:
        # Level 1: outer ∘ middle
        level1 = ConcatenatedTopologicalCSSCode(outer_code, middle_code)
        
        # Level 2: (outer ∘ middle) ∘ inner
        level2 = ConcatenatedTopologicalCSSCode(level1, inner_code)
        
        result['n'] = level2.n
        result['k'] = level2.k
        
        # Verify expected n
        expected_n = outer_code.n * middle_code.n * inner_code.n
        if level2.n == expected_n and level2.k > 0:
            result['status'] = 'OK'
        else:
            result['status'] = 'WARN'
            result['error'] = f"n={level2.n} (expected {expected_n}), k={level2.k}"
            
    except Exception as e:
        result['status'] = 'FAIL'
        result['error'] = f"{type(e).__name__}: {str(e)[:30]}"
    
    multi_results.append(result)
    
    if result['status'] == 'OK':
        print(f"{STATUS_OK} n={result['n']}, k={result['k']}")
    elif result['status'] == 'WARN':
        print(f"{STATUS_WARN} {result['error']}")
    else:
        print(f"{STATUS_FAIL} {result['error']}")

# Summary
if multi_results:
    ok = sum(1 for r in multi_results if r['status'] == 'OK')
    warn = sum(1 for r in multi_results if r['status'] == 'WARN')
    fail = sum(1 for r in multi_results if r['status'] == 'FAIL')
    print("\n" + "-"*70)
    print(f"MULTI-LEVEL SUMMARY: {ok} {STATUS_OK} | {warn} {STATUS_WARN} | {fail} {STATUS_FAIL}")

MULTI-LEVEL CONCATENATION TESTS

Testing nested concatenation: (outer ∘ middle) ∘ inner
Demonstrates composability of composite operations

Testing 3 multi-level concatenations...

[1/3] (FourQubit422_[[4,2,2]] ∘ C6) ∘ Steane_713... ✓ n=168, k=6
[2/3] (C6 ∘ Steane_713) ∘ Shor_91... ✓ n=168, k=6
[2/3] (C6 ∘ Steane_713) ∘ Shor_91... ✓ n=378, k=2
[3/3] (Steane_713 ∘ Shor_91) ∘ FourQubit422_[[4,2,2]]... ✓ n=378, k=2
[3/3] (Steane_713 ∘ Shor_91) ∘ FourQubit422_[[4,2,2]]... ✓ n=252, k=64

----------------------------------------------------------------------
MULTI-LEVEL SUMMARY: 3 ✓ | 0 ⚠️ | 0 ✗
✓ n=252, k=64

----------------------------------------------------------------------
MULTI-LEVEL SUMMARY: 3 ✓ | 0 ⚠️ | 0 ✗


In [16]:
"""Cell 6: Decoder Smoke Test on Composite Codes - All Decoders"""
clear_output(wait=True)

print("="*120)
print("DECODER SMOKE TEST ON COMPOSITE CODES - ALL DECODERS")
print("="*120)

if len(decoder_classes) == 0:
    print("\n⚠️ No decoders available - skipping decoder tests")
else:
    # Select a subset of decoders for faster testing
    # Focus on the most reliable ones
    test_decoders = ['PyMatching', 'FusionBlossom', 'BPOSD', 'Tesseract']
    test_decoders = [d for d in test_decoders if d in decoder_classes]
    
    print(f"\nTesting {len(test_decoders)} decoders on successful concatenated codes")
    print(f"Decoders: {test_decoders}\n")
    
    # Select successful concatenations (limit to 3 for speed)
    successful_concats = [r for r in concat_results if r['status'] == 'OK'][:3]
    
    if len(successful_concats) == 0:
        print("⚠️ No successful concatenations to test")
    else:
        decoder_results = []
        
        # Header
        header = f"{'Composite Code':<35}"
        for dec_name in test_decoders:
            header += f" | {dec_name[:12]:>12}"
        header += " | No-decode"
        print(header)
        print("-" * len(header))
        
        for concat_res in successful_concats:
            outer_name = concat_res['outer']
            inner_name = concat_res['inner']
            code_label = f"{outer_name[:15]}∘{inner_name[:15]}"
            
            row = f"{code_label:<35}"
            ler_no_decode = None
            
            try:
                # Rebuild the concatenated code
                outer_code = next((c for n, c in css_codes.items() if n == outer_name), None)
                inner_code = next((c for n, c in css_codes.items() if n == inner_name), None)
                
                if outer_code is None or inner_code is None:
                    row += " | Code not found"
                    print(row)
                    continue
                    
                concat = ConcatenatedTopologicalCSSCode(outer_code, inner_code)
                
                for dec_name in test_decoders:
                    try:
                        test_res = test_decoder_on_code(
                            concat,
                            decoder_classes[dec_name],
                            decoder_name=dec_name,
                            code_type='CSS',
                            p=0.01,
                            shots=500,
                            rounds=2,
                        )
                        
                        if ler_no_decode is None and test_res.ler_no_decode is not None:
                            ler_no_decode = test_res.ler_no_decode
                        
                        if test_res.status == 'OK':
                            ler_str = f"✓{test_res.ler:.4f}" if test_res.ler else "✓N/A"
                        elif test_res.status == 'SKIP':
                            ler_str = "-SKIP"
                        else:
                            ler_str = "✗FAIL"
                            
                        row += f" | {ler_str:>12}"
                        
                        decoder_results.append({
                            'code': code_label,
                            'decoder': dec_name,
                            'status': test_res.status,
                            'ler': test_res.ler,
                        })
                        
                    except Exception as e:
                        row += f" | {'✗ERR':>12}"
                        decoder_results.append({
                            'code': code_label,
                            'decoder': dec_name,
                            'status': 'FAIL',
                            'error': str(e)[:30],
                        })
                
                nd_str = f"{ler_no_decode:.4f}" if ler_no_decode else "N/A"
                row += f" | {nd_str:>10}"
                        
            except Exception as e:
                row += f" | Build error: {type(e).__name__}"
            
            print(row)
        
        print("-" * len(header))
        
        # Summary
        if decoder_results:
            ok = sum(1 for r in decoder_results if r.get('status') == 'OK')
            skip = sum(1 for r in decoder_results if r.get('status') == 'SKIP')
            fail = sum(1 for r in decoder_results if r.get('status') not in ['OK', 'SKIP'])
            print(f"\nDECODER SUMMARY: {ok} {STATUS_OK} | {skip} {STATUS_SKIP} | {fail} {STATUS_FAIL}")

DECODER SMOKE TEST ON COMPOSITE CODES - ALL DECODERS

Testing 4 decoders on successful concatenated codes
Decoders: ['PyMatching', 'FusionBlossom', 'BPOSD', 'Tesseract']

Composite Code                      |   PyMatching | FusionBlosso |        BPOSD |    Tesseract | No-decode
-----------------------------------------------------------------------------------------------------------
RotatedSurface_∘FourQubit422_[[     |      ✓0.0040 |      ✓0.1820 |      ✓0.0020 |         ✓N/A |     0.1520
RotatedSurface_∘FourQubit422_[[     |      ✓0.0040 |      ✓0.1820 |      ✓0.0020 |         ✓N/A |     0.1520
RotatedSurface_∘C6                  |      ✓0.0060 |      ✓0.1880 |         ✓N/A |      ✓0.0100 |     0.1660
RotatedSurface_∘C6                  |      ✓0.0060 |      ✓0.1880 |         ✓N/A |      ✓0.0100 |     0.1660
RotatedSurface_∘Steane_713          |      ✓0.2320 |      ✓0.2740 |      ✓0.0040 |      ✓0.0020 |     0.2680
--------------------------------------------------------------------

In [17]:
"""Cell 7: Dual Code Tests"""
clear_output(wait=True)

print("="*70)
print("DUAL CODE TESTS")
print("="*70)
print("\nTesting DualCode(code) - swaps X and Z sectors\n")

dual_results = []

# Test on a selection of codes
dual_test_codes = list(css_codes.items())[:10]

for i, (name, code) in enumerate(dual_test_codes):
    print(f"[{i+1}/{len(dual_test_codes)}] Dual({name})...", end=' ', flush=True)
    
    result = {
        'original': name,
        'status': 'UNKNOWN',
        'n': None,
        'k': None,
        'error': None,
    }
    
    try:
        dual = DualCode(code)
        result['n'] = dual.n
        result['k'] = dual.k
        
        # Verify n and k preserved
        if dual.n == code.n and dual.k == code.k:
            result['status'] = 'OK'
        else:
            result['status'] = 'WARN'
            result['error'] = f"n,k changed: {code.n},{code.k} -> {dual.n},{dual.k}"
            
    except Exception as e:
        result['status'] = 'FAIL'
        result['error'] = f"{type(e).__name__}: {str(e)[:30]}"
    
    dual_results.append(result)
    
    if result['status'] == 'OK':
        print(f"{STATUS_OK} n={result['n']}, k={result['k']}")
    elif result['status'] == 'WARN':
        print(f"{STATUS_WARN} {result['error']}")
    else:
        print(f"{STATUS_FAIL} {result['error']}")

# Summary
ok = sum(1 for r in dual_results if r['status'] == 'OK')
warn = sum(1 for r in dual_results if r['status'] == 'WARN')
fail = sum(1 for r in dual_results if r['status'] == 'FAIL')

print("\n" + "-"*70)
print(f"DUAL CODE SUMMARY: {ok} {STATUS_OK} | {warn} {STATUS_WARN} | {fail} {STATUS_FAIL}")

DUAL CODE TESTS

Testing DualCode(code) - swaps X and Z sectors

[1/10] Dual(FourQubit422_[[4,2,2]])... ✓ n=4, k=2
[2/10] Dual(C6)... ✓ n=6, k=2
[3/10] Dual(Steane_713)... ✓ n=7, k=1
[4/10] Dual(Shor_91)... ✓ n=9, k=1
[5/10] Dual(ReedMuller_15_1_3)... ✓ n=15, k=1
[6/10] Dual(Hamming_CSS_7)... ✓ n=7, k=1
[7/10] Dual(Code_832)... ✓ n=8, k=3
[8/10] Dual(Repetition_3)... ✓ n=3, k=1
[9/10] Dual(Repetition_5)... ✓ n=5, k=1
[10/10] Dual(Repetition_7)... ✓ n=7, k=1

----------------------------------------------------------------------
DUAL CODE SUMMARY: 10 ✓ | 0 ⚠️ | 0 ✗
✓ n=4, k=2
[2/10] Dual(C6)... ✓ n=6, k=2
[3/10] Dual(Steane_713)... ✓ n=7, k=1
[4/10] Dual(Shor_91)... ✓ n=9, k=1
[5/10] Dual(ReedMuller_15_1_3)... ✓ n=15, k=1
[6/10] Dual(Hamming_CSS_7)... ✓ n=7, k=1
[7/10] Dual(Code_832)... ✓ n=8, k=3
[8/10] Dual(Repetition_3)... ✓ n=3, k=1
[9/10] Dual(Repetition_5)... ✓ n=5, k=1
[10/10] Dual(Repetition_7)... ✓ n=7, k=1

----------------------------------------------------------------------

In [18]:
"""Cell 8: Comprehensive Results Table"""
clear_output(wait=True)

print("="*100)
print("COMPOSITE CODE SMOKE TEST - COMPREHENSIVE RESULTS")
print("="*100)

# Concatenation results
print("\nCONCATENATION (ConcatenatedTopologicalCSSCode):")
print("-"*100)
print(f"{'Outer':<20} | {'Inner':<20} | {'n':>5} | {'k':>3} | {'Circuit':>8} | Status")
print("-"*100)

for r in concat_results:
    outer = r['outer'][:20] if r['outer'] else 'N/A'
    inner = r['inner'][:20] if r['inner'] else 'N/A'
    n = str(r['n']) if r['n'] else '-'
    k = str(r['k']) if r['k'] else '-'
    circ = STATUS_OK if r.get('circuit_ok') else STATUS_FAIL
    
    if r['status'] == 'OK':
        status = f"{STATUS_OK} OK"
    elif r['status'] == 'WARN':
        status = f"{STATUS_WARN} {r['error'][:20]}" if r['error'] else f"{STATUS_WARN}"
    else:
        status = f"{STATUS_FAIL} {r['error'][:20]}" if r['error'] else f"{STATUS_FAIL}"
    
    print(f"{outer:<20} | {inner:<20} | {n:>5} | {k:>3} | {circ:>8} | {status}")

# Multi-level results
if multi_results:
    print("\n\nMULTI-LEVEL CONCATENATION:")
    print("-"*100)
    for r in multi_results:
        desc = r['desc'][:50] if r.get('desc') else 'N/A'
        n = str(r['n']) if r['n'] else '-'
        k = str(r['k']) if r['k'] else '-'
        
        if r['status'] == 'OK':
            status = f"{STATUS_OK} OK"
        elif r['status'] == 'WARN':
            status = f"{STATUS_WARN}"
        else:
            status = f"{STATUS_FAIL} {r['error'][:30]}" if r['error'] else f"{STATUS_FAIL}"
        
        print(f"{desc:<50} | n={n:>5} k={k:>3} | {status}")

# Homological product results
if hom_results:
    print("\n\nHOMOLOGICAL PRODUCT:")
    print("-"*100)
    for r in hom_results:
        code_a = r['code_a'][:20] if r.get('code_a') else 'N/A'
        code_b = r['code_b'][:20] if r.get('code_b') else 'N/A'
        n = str(r['n']) if r['n'] else '-'
        k = str(r['k']) if r['k'] else '-'
        
        if r['status'] == 'OK':
            status = f"{STATUS_OK} OK"
        else:
            status = f"{STATUS_FAIL} {r['error'][:30]}" if r['error'] else f"{STATUS_FAIL}"
        
        print(f"{code_a:<20} ⊗ {code_b:<20} | n={n:>5} k={k:>3} | {status}")

# Dual code results
if dual_results:
    print("\n\nDUAL CODES:")
    print("-"*100)
    for r in dual_results:
        orig = r['original'][:30] if r.get('original') else 'N/A'
        n = str(r['n']) if r['n'] else '-'
        k = str(r['k']) if r['k'] else '-'
        
        if r['status'] == 'OK':
            status = f"{STATUS_OK}"
        elif r['status'] == 'WARN':
            status = f"{STATUS_WARN}"
        else:
            status = f"{STATUS_FAIL}"
        
        print(f"Dual({orig:<30}) | n={n:>5} k={k:>3} | {status}")

COMPOSITE CODE SMOKE TEST - COMPREHENSIVE RESULTS

CONCATENATION (ConcatenatedTopologicalCSSCode):
----------------------------------------------------------------------------------------------------
Outer                | Inner                |     n |   k |  Circuit | Status
----------------------------------------------------------------------------------------------------
RotatedSurface_[[9,1 | FourQubit422_[[4,2,2 |    36 |  10 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | C6                   |    54 |  10 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Steane_713           |    63 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Shor_91              |    81 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Hamming_CSS_7        |    63 |   1 |        ✓ | ✓ OK
RotatedSurface_[[25, | FourQubit422_[[4,2,2 |   100 |  26 |        ✓ | ✓ OK
RotatedSurface_[[25, | C6                   |   150 |  26 |        ✓ | ✓ OK
RotatedSurface_[[25, | Steane_713           |   175 |   1 |        ✓ | ✓ OK
RotatedSurfac

In [19]:
"""Cell 9: Final Summary"""
print("\n" + "="*70)
print("COMPOSITE CODE SMOKE TEST - FINAL SUMMARY")
print("="*70)

# Count totals
concat_ok = sum(1 for r in concat_results if r['status'] == 'OK')
concat_total = len(concat_results)

multi_ok = sum(1 for r in multi_results if r['status'] == 'OK') if multi_results else 0
multi_total = len(multi_results) if multi_results else 0

hom_ok = sum(1 for r in hom_results if r['status'] == 'OK') if hom_results else 0
hom_total = len(hom_results) if hom_results else 0

dual_ok = sum(1 for r in dual_results if r['status'] == 'OK') if dual_results else 0
dual_total = len(dual_results) if dual_results else 0

print(f"\n{'Operation':<30} | {'Passed':>8} | {'Total':>8} | {'Rate':>8}")
print("-"*60)
print(f"{'Concatenation':<30} | {concat_ok:>8} | {concat_total:>8} | {100*concat_ok/concat_total if concat_total else 0:.0f}%")
print(f"{'Multi-level Concatenation':<30} | {multi_ok:>8} | {multi_total:>8} | {100*multi_ok/multi_total if multi_total else 0:.0f}%")
print(f"{'Homological Product':<30} | {hom_ok:>8} | {hom_total:>8} | {100*hom_ok/hom_total if hom_total else 0:.0f}%")
print(f"{'Dual Code':<30} | {dual_ok:>8} | {dual_total:>8} | {100*dual_ok/dual_total if dual_total else 0:.0f}%")
print("-"*60)

total_ok = concat_ok + multi_ok + hom_ok + dual_ok
total_tests = concat_total + multi_total + hom_total + dual_total

print(f"{'TOTAL':<30} | {total_ok:>8} | {total_tests:>8} | {100*total_ok/total_tests if total_tests else 0:.0f}%")

print("\n" + "="*70)
print(f"Building block codes used: {len(css_codes)}")
print(f"Composite constructions tested: {total_tests}")

if total_ok == total_tests:
    print(f"\n{STATUS_OK} ALL COMPOSITE TESTS PASSED")
else:
    print(f"\n{STATUS_WARN} {total_tests - total_ok} tests need attention")
print("="*70)


COMPOSITE CODE SMOKE TEST - FINAL SUMMARY

Operation                      |   Passed |    Total |     Rate
------------------------------------------------------------
Concatenation                  |       14 |       24 | 58%
Multi-level Concatenation      |        3 |        3 | 100%
Homological Product            |       10 |       10 | 100%
Dual Code                      |       10 |       10 | 100%
------------------------------------------------------------
TOTAL                          |       37 |       47 | 79%

Building block codes used: 33
Composite constructions tested: 47

⚠️ 10 tests need attention
